In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# **Carga de datos para el fit**

In [ ]:
train = pd.read_csv('data/train.csv')
data = train.copy()


# **Preprocesado del DataFrame**
Usamos imputación con la estrategia adecuada para cada columna

# **Imputación para columnas numéricas**




Esta función transforma valores categóricos relacionados con los rangos de matrícula universitaria a valores numéricos promedio.

In [ ]:
def convertir_valor_matricula(valor):
    if pd.isna(valor):
        return np.nan
    elif 'Menos de 500 mil' in valor:
        return 250000  # Asignamos un valor promedio entre 0 y 500 mil
    elif 'Entre 500 mil y menos de 1 millón' in valor:
        return 750000  # Promedio entre 500 mil y 1 millón
    elif 'Entre 1 millón y menos de 2.5 millones' in valor:
        return 1750000  # Promedio entre 1 y 2.5 millones
    elif 'Entre 2.5 millones y menos de 4 millones' in valor:
        return 3250000  # Promedio entre 2.5 y 4 millones
    elif 'Entre 4 millones y menos de 5.5 millones' in valor:
        return 4750000  # Promedio entre 4 y 5.5 millones
    elif 'Entre 5.5 millones y menos de 7 millones' in valor:
        return 6250000  # Promedio entre 5.5 y 7 millones
    elif 'Más de 7 millones' in valor:
        return 7500000  # Asignamos un valor mínimo representativo superior a 7 millones
    elif 'No pagó matrícula' in valor:
        return 0  # Asumimos que no se pagó nada
    else:
        return np.nan  # Para cualquier caso que no coincida

Esta función transforma valores categóricos que representan rangos de horas trabajadas semanalmente en valores numéricos promedio o representativos.

In [ ]:
def convertir_horas_trabajadas(valor):
    if isinstance(valor, str):
        if "Entre" in valor:
            partes = valor.split('y')
            min_val = float(partes[0].split(' ')[-2])  # Obtener el penúltimo elemento
            max_val = float(partes[1].split(' ')[-2])  # Obtener el penúltimo elemento de la segunda parte
            return (min_val + max_val) / 2  # Retornar el promedio del rango

        elif "Más de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "Menos de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "0" in valor:
            return 0  # Devolver 0 en número

    return np.nan  # Devolver NaN si no es un valor válido

## **Aplicación de funciones e imputaciones para columnas numéricas**

In [ ]:
# Aplicar las funciones de conversión
data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)

data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

# Imputar valores faltantes
num_imputer = SimpleImputer(strategy='mean')

data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']] = num_imputer.fit_transform(
    data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']]
)

# Agregar num_columns
num_columns = data.select_dtypes(include=['number']).columns
num_columns = [col for col in num_columns if col not in ['ID']]

# **Imputación para columnas categóricas**

Esta celda utiliza la técnica de imputación para manejar valores faltantes en columnas categóricas. Específicamente, aplica el método de la moda para rellenar los valores faltantes en las columnas indicadas.

In [ ]:
cat_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_ESTRATOVIVIENDA',
               'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']

cat_imputer = SimpleImputer(strategy='most_frequent')  # Usamos la moda para imputar valores categóricos


data[cat_columns] = cat_imputer.fit_transform(data[cat_columns])


# **2. Conversión de variables categóricas**
Esta celda transforma columnas categóricas binarias en un formato numérico (1 y 0). Específicamente, mapea los valores 'Si' y 'No' en las columnas indicadas por la lista binary_columns.

In [ ]:
binary_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']


def map_binary_columns(data, binary_columns, mapping={'Si': 1, 'No': 0}):
    for col in binary_columns:
        data[col] = data[col].map(mapping)
    return data

data = map_binary_columns(data, binary_columns)

**Codificación de variables categóricas multiclase con diccionario**

Esta celda transforma los valores categóricos de la columna FAMI_ESTRATOVIVIENDA, que representan estratos socioeconómicos en formato de texto (por ejemplo, 'Estrato 1'), en valores numéricos. Adicionalmente, se maneja el valor 'Sin Estrato' y los valores faltantes (NaN) asignándoles un valor de 0.



In [ ]:
# Crear un diccionario para asignar el valor numérico a cada valor en letra de la columna FAMI_ESTRATOVIVIENDA
moda_estrato = data['FAMI_ESTRATOVIVIENDA'].mode()[0]

estrato_dict = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0,
    np.nan: 0
}

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


<ipython-input-63-bb163e89f875>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


Esta celda transforma los valores categóricos en la columna RENDIMIENTO_GLOBAL, que representan niveles de rendimiento (por ejemplo, 'bajo', 'alto'), en valores numéricos mediante un mapeo definido en un diccionario.

In [ ]:
rendimiento_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].map(rendimiento_mapping)

Esta celda aplica codificación de etiquetas (Label Encoding) a las columnas categóricas especificadas en la lista label_columns. Este proceso convierte los valores categóricos en números enteros secuenciales. Adicionalmente, almacena los codificadores (LabelEncoder) utilizados, permitiendo reutilizarlos posteriormente para transformar datos nuevos o invertir el proceso.


In [ ]:
label_columns = ['ESTU_PRGM_ACADEMICO']

def apply_label_encoding(data, columns, label_encoders={}):
    for col in columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le
    return data, label_encoders

data, label_encoders = apply_label_encoding(data, label_columns)

# **3. One-Hot Encoding para variables indicadoras (educación de padres y madres)**

Esta celda implementa una función para aplicar codificación One-Hot Encoding a columnas categóricas especificadas. One-Hot Encoding convierte cada categoría en una columna binaria, donde un valor de 1 indica la presencia de la categoría y un valor de 0 su ausencia. La función también maneja valores faltantes imputándolos con un valor predeterminado.

In [ ]:

columns_to_encode = ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']

def apply_onehot_encoding(data, columns, fill_value='No Aplica'):
  onehot_encoders = {}

  for col in columns:
        data[col].fillna(fill_value, inplace=True)
        unique_values = sorted(data[col].unique())
        onehot_encoders[col] = unique_values
        onehot_data = pd.get_dummies(data[col], prefix=col)
        data = pd.concat([data, onehot_data], axis=1)
        data.drop(col, axis=1, inplace=True)
  return data, onehot_encoders

data, onehot_encoders = apply_onehot_encoding(data, columns_to_encode)

<ipython-input-66-92176bd53bb2>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(fill_value, inplace=True)
<ipython-input-66-92176bd53bb2>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

# **4. Escalado de variables numéricas**
Esta celda normaliza las columnas numéricas y otras seleccionadas del DataFrame utilizando el escalador MinMaxScaler de scikit-learn. La normalización transforma los valores de las columnas a un rango específico (en este caso, de -1 a 1).

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
data[num_columns] = scaler.fit_transform(data[num_columns])

data['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(data[['ESTU_PRGM_ACADEMICO']])
data['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(data[['FAMI_ESTRATOVIVIENDA']])

# ***Guardamos el preprocesamiento en un diccionario para usarlo en el conjunto de prueba***



---



In [ ]:
preprocessing_objects = {
    'num_imputer': num_imputer,
    'cat_imputer': cat_imputer,
    'label_encoders': label_encoders,
    'onehot_encoders': onehot_encoders,
}

# **Dataset preprocesado**

In [ ]:
data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,FAMI_EDUCACIONMADRE_Educación profesional completa,...,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,ESTU_HORASSEMANATRABAJA_0.0,ESTU_HORASSEMANATRABAJA_10.0,ESTU_HORASSEMANATRABAJA_15.5,ESTU_HORASSEMANATRABAJA_18.910386719121945,ESTU_HORASSEMANATRABAJA_25.5,ESTU_HORASSEMANATRABAJA_30.0
0,904256,0.933333,-0.366420,BOGOTÁ,0.666667,0.000000,1,0,2,False,...,False,False,False,True,False,True,False,False,False,False
1,645256,0.933333,-0.474129,ATLANTICO,-0.133333,0.000000,0,0,0,False,...,False,False,True,False,True,False,False,False,False,False
2,308367,0.333333,0.729673,BOGOTÁ,-0.133333,0.000000,1,0,0,False,...,True,False,False,False,False,False,False,False,False,True
3,470353,-0.200000,-0.970433,SANTANDER,0.266667,0.333333,1,0,3,False,...,False,False,False,False,True,False,False,False,False,False
4,989032,0.933333,0.915523,ANTIOQUIA,-0.133333,0.000000,1,0,1,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,-0.200000,-0.695882,LA GUAJIRA,-0.800000,-0.333333,1,1,2,False,...,True,False,False,False,False,False,True,False,False,False
692496,754213,0.933333,0.915523,NORTE SANTANDER,-0.133333,0.000000,1,0,0,False,...,False,False,False,False,False,False,False,False,False,True
692497,504185,-1.000000,-0.801478,BOGOTÁ,-0.533333,0.000000,1,1,1,False,...,True,False,False,False,False,True,False,False,False,False
692498,986620,-0.200000,0.915523,TOLIMA,-0.133333,-0.666667,0,1,0,False,...,False,False,False,False,False,True,False,False,False,False


# **Entrenamiento**

Esta celda entrena un modelo de clasificación (LGBMClassifier) para predecir el rendimiento global de estudiantes (RENDIMIENTO_GLOBAL) por departamento (ESTU_PRGM_DEPARTAMENTO). Los modelos se entrenan de forma individual para cada departamento, ordenados por el promedio del estrato socioeconómico (FAMI_ESTRATOVIVIENDA) en cada departamento.

In [ ]:
# Ordenar departamentos por estrato promedio
dept_estrato = data.groupby('ESTU_PRGM_DEPARTAMENTO')['FAMI_ESTRATOVIVIENDA'].mean().sort_values()
departments = dept_estrato.index.tolist()

# Inicializar diccionario para almacenar los modelos por departamento
models_by_department = {}

# Entrenamiento para cada departamento
for department in departments:
  print(f"---{department}---")
  # Filtrar datos del departamento
  dept_data = data[data['ESTU_PRGM_DEPARTAMENTO'] == department]

  # Separar características y etiqueta
  X_train = dept_data.drop(columns=['RENDIMIENTO_GLOBAL', 'ID', 'ESTU_PRGM_DEPARTAMENTO'])
  y_train = dept_data['RENDIMIENTO_GLOBAL']


  # Crear y entrenar el modelo
  model = LGBMClassifier(
    random_state=71,  # Semilla para reproducibilidad
    objective='multiclass',  # Configuración para multiclase
    num_class=4,  # Número de clases
    boosting_type='gbdt',  # Tipo de boosting (Gradient Boosting Decision Trees)
    learning_rate=0.05,  # Tasa de aprendizaje
    n_estimators=500,  # Número de iteraciones (árboles)
    max_depth=10,  # Profundidad máxima del árbol
    num_leaves=31,  # Número de hojas máximo por árbol (2^max_depth - 1 es el límite superior)
    subsample=0.8,  # Proporción de datos usados en cada iteración
    colsample_bytree=0.7,  # Proporción de características usadas por árbol
    reg_lambda=1.0,  # Regularización L2
    reg_alpha=0.1,  # Regularización L1
    min_child_weight=3,  # Peso mínimo en hojas
    min_split_gain=0.1,  # Ganancia mínima para dividir un nodo
    importance_type='gain',  # Métrica de importancia para características
    verbosity=-1
)



  model.fit(X_train, y_train)
  y_val_pred = model.predict(X_train)
  accuracy = accuracy_score(y_train, y_val_pred)
  print(f"Reporte de clasificación para {department}:" + f"{accuracy}")


   # Guardar el modelo entrenado por departamento
  models_by_department[department] = model


---PUTUMAYO---
Reporte de clasificación para PUTUMAYO:0.5949685534591195
---CHOCO---
Reporte de clasificación para CHOCO:0.7684775005828864
---GUAVIARE---
Reporte de clasificación para GUAVIARE:0.5135135135135135
---LA GUAJIRA---
Reporte de clasificación para LA GUAJIRA:0.6161573880284638
---ARAUCA---
Reporte de clasificación para ARAUCA:0.5891341256366723
---CAQUETA---
Reporte de clasificación para CAQUETA:0.560737119217751
---AMAZONAS---
Reporte de clasificación para AMAZONAS:0.425
---CORDOBA---
Reporte de clasificación para CORDOBA:0.49868723334427306
---SUCRE---
Reporte de clasificación para SUCRE:0.5213281069863961
---SAN ANDRES---
Reporte de clasificación para SAN ANDRES:0.6
---CASANARE---
Reporte de clasificación para CASANARE:0.5599352051835853
---CESAR---
Reporte de clasificación para CESAR:0.4931755042879575
---NARIÑO---
Reporte de clasificación para NARIÑO:0.49576334175709824
---HUILA---
Reporte de clasificación para HUILA:0.5169584792396198
---NORTE SANTANDER---
Reporte de 

KeyboardInterrupt: 

# **Carga de datos para el Predict**

In [ ]:

test = pd.read_csv('data/test.csv')
test_data = test.copy()

# **Preprocesamiento del conjunto de prueba**

Esta celda aplica varias etapas de preprocesamiento al conjunto de prueba (test_data) para asegurar que esté en el mismo formato que los datos utilizados para entrenar los modelos. El objetivo es garantizar la consistencia entre las características del conjunto de prueba y el modelo.

In [ ]:
# Eliminar columna innecesaria
test_data = test_data.drop('Unnamed: 0', axis=1)

# Transformar valores inconsistentes en columnas específicas
test_data['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_data['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)
test_data['ESTU_HORASSEMANATRABAJA'] = test_data['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

# Imputar valores faltantes en columnas numéricas
test_data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']] = num_imputer.fit_transform(
    test_data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']]
)

# Escalar columnas numéricas al rango [-1, 1]
num_columns = test_data.select_dtypes(include=['number']).columns
num_columns = [col for col in num_columns if col not in ['ID']]
test_data[num_columns] = scaler.fit_transform(test_data[num_columns])

# Imputar valores faltantes en columnas categóricas
cat_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET']
test_data[cat_columns] = cat_imputer.fit_transform(test_data[cat_columns])

# Mapear valores de FAMI_ESTRATOVIVIENDA a valores numéricos
test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)

# Transformar columnas binarias a valores numéricos
binary_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']
test_data = map_binary_columns(test_data, binary_columns)

# Aplicar One-Hot Encoding a columnas específicas
columns_to_encode = ['FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE']
test_data, onehot_encoders = apply_onehot_encoding(test_data, columns_to_encode)

# Aplicar Label Encoding a columnas específicas
label_columns = ['ESTU_PRGM_ACADEMICO']
test_data, label_encoders = apply_label_encoding(test_data, label_columns)

# Escalar columnas específicas nuevamente
test_data['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(test_data[['ESTU_PRGM_ACADEMICO']])
test_data['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(test_data[['FAMI_ESTRATOVIVIENDA']])


<ipython-input-264-34860d1c1007>:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)
<ipython-input-257-780daa6885c1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(fill_value, inplace=True)


In [ ]:
test_data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONMADRE_Educación profesional completa,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,550236,-1.000000,TRABAJO SOCIAL,BOLIVAR,-0.933333,-0.333333,0.000000,1,1.0,False,...,False,False,False,False,False,False,False,False,True,False
1,98545,0.333333,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,-0.133333,0.700000,-0.333333,1,0.0,False,...,False,False,False,False,False,False,True,False,False,False
2,499179,0.933333,INGENIERIA MECATRONICA,BOGOTÁ,-0.533333,-1.000000,0.000000,1,0.0,False,...,False,False,False,False,False,False,False,True,False,False
3,782980,-0.200000,CONTADURIA PUBLICA,SUCRE,-0.533333,0.700000,-0.666667,0,0.0,False,...,False,False,False,False,False,True,False,False,False,False
4,785185,0.933333,ADMINISTRACION DE EMPRESAS,ATLANTICO,-0.133333,0.033333,-0.333333,1,0.0,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,-0.200000,ADMINISTRACION DE EMPRESAS,BOGOTÁ,-0.133333,1.000000,-0.666667,1,1.0,False,...,False,False,False,False,False,True,False,False,False,False
296782,209415,-1.000000,DERECHO,META,-0.533333,-1.000000,0.333333,1,0.0,True,...,False,False,False,False,False,False,False,False,False,False
296783,239074,0.933333,DERECHO,BOGOTÁ,-0.133333,1.000000,0.000000,1,0.0,True,...,False,False,False,False,False,False,True,False,False,False
296784,963852,-0.200000,INGENIERIA AERONAUTICA,ANTIOQUIA,0.666667,0.033333,0.000000,1,0.0,True,...,False,False,False,False,False,False,False,False,False,False


# **Predicción**
Esta celda utiliza los modelos entrenados para realizar predicciones del rendimiento global de estudiantes en un conjunto de prueba (test_data). Además, maneja casos en los que no hay un modelo entrenado para un departamento específico, asignando predicciones por defecto. Finalmente, los resultados se guardan en un archivo CSV para ser envaluados en Kaggle.

In [ ]:
results = []

# Predecir para cada departamento
for department, model in models_by_department.items():
    print(f"Realizando predicciones para el departamento: {department}")

# Filtrar los datos del test para el departamento
    test_dept_data = test_data[test_data['ESTU_PRGM_DEPARTAMENTO'] == department]

    if test_dept_data.empty:
        continue

# Separar las características del conjunto de prueba
    X_test = test_dept_data.drop(columns=['ID', 'ESTU_PRGM_DEPARTAMENTO'], errors='ignore')

# Realizar predicción
    y_test_pred = model.predict(X_test)

# Convertir las predicciones a etiquetas de rendimiento
    y_test_pred_labels = [list(rendimiento_mapping.keys())[pred] for pred in y_test_pred]

# Guardar los resultados
    results.extend(zip(test_dept_data['ID'], y_test_pred_labels))

# Manejo de departamentos no vistos

unseen_departments = set(test['ESTU_PRGM_DEPARTAMENTO'].unique()) - set(models_by_department.keys())
for department in unseen_departments:
        print(f"No hay datos de entrenamiento para el departamento: {department}. Asignando predicciones globales.")
        test_dept_data = test_data[test_data['ESTU_PRGM_DEPARTAMENTO'] == department]
        y_test_pred_labels = ['medio-bajo'] * len(test_dept_data)  # Predicción por defecto
        results.extend(zip(test_dept_data['ID'], y_test_pred_labels))

# Crear archivo de envío
submission = pd.DataFrame(results, columns=['ID', 'RENDIMIENTO_GLOBAL'])
submission.to_csv('submission-IA.csv', index=False)
print("Archivo de predicciones creado: 'submission.csv'")

Realizando predicciones para el departamento: PUTUMAYO
Realizando predicciones para el departamento: CHOCO
Realizando predicciones para el departamento: GUAVIARE
Realizando predicciones para el departamento: LA GUAJIRA
Realizando predicciones para el departamento: ARAUCA
Realizando predicciones para el departamento: CAQUETA
Realizando predicciones para el departamento: AMAZONAS
Realizando predicciones para el departamento: CORDOBA
Realizando predicciones para el departamento: SUCRE
Realizando predicciones para el departamento: SAN ANDRES
Realizando predicciones para el departamento: CASANARE
Realizando predicciones para el departamento: CESAR
Realizando predicciones para el departamento: NARIÑO
Realizando predicciones para el departamento: HUILA
Realizando predicciones para el departamento: NORTE SANTANDER
Realizando predicciones para el departamento: CAUCA
Realizando predicciones para el departamento: VAUPES
Realizando predicciones para el departamento: MAGDALENA
Realizando prediccion

Esta celda carga el archivo de predicciones generado (submission-IA.csv) y analiza la distribución de las etiquetas de rendimiento global predichas

In [ ]:
sub = pd.read_csv('submission-IA.csv')
sub['RENDIMIENTO_GLOBAL'].value_counts()

,count
RENDIMIENTO_GLOBAL,
bajo,93386
alto,88024
medio-bajo,59721
medio-alto,55655
